In [1]:
from func_def import *

In [4]:
events = get_links(url='http://www.ufcstats.com/statistics/events/completed?page=all')
events.date = pd.to_datetime(events.date)
events = events[events.date < get_current_date()].reset_index(drop=True)
events.to_csv('event_list.csv')
events

,date,event_url
0,2023-10-07,http://www.ufcstats.com/event-details/c8a49ff2...
1,2023-09-23,http://www.ufcstats.com/event-details/c945adc2...
2,2023-09-16,http://www.ufcstats.com/event-details/8fa2b065...
3,2023-09-09,http://www.ufcstats.com/event-details/ece28074...
4,2023-09-02,http://www.ufcstats.com/event-details/ef61d9f5...
...,...,...
660,1995-07-14,http://www.ufcstats.com/event-details/1c3f5e85...
661,1995-04-07,http://www.ufcstats.com/event-details/dedc3bb4...
662,1994-12-16,http://www.ufcstats.com/event-details/b60391da...
663,1994-09-09,http://www.ufcstats.com/event-details/1a49e067...


In [5]:
#get existing data
UPDATE_ALL = True

exist_data = []
exist_events_url = []

if not UPDATE_ALL:
    try:
        exist_data = pd.read_csv('data_pre_processed.csv',index_col=0)
        exist_events_url = list(np.unique(exist_data.event_url))
    except:
        print('Cant find data_pre_processed.csv')
        exist_data = []
        exist_events_url = []

print(f'Current Existing events: {len(exist_events_url)}')

Current Existing events: 0


In [6]:
#get new data
new_events = events[~events.event_url.isin(exist_events_url)].reset_index(drop=True)
#new_events = events
print(f'New events: {len(new_events)}')
new_events.head(3)

New events: 665


,date,event_url
0,2023-10-07,http://www.ufcstats.com/event-details/c8a49ff2...
1,2023-09-23,http://www.ufcstats.com/event-details/c945adc2...
2,2023-09-16,http://www.ufcstats.com/event-details/8fa2b065...


In [7]:
new_data = []
for index in tqdm(range(len(new_events))):
    try:
        new_data.extend(get_event_data(EVENT_URL=new_events.loc[index,'event_url'],EVENT_TIME=new_events.loc[index,'date']))
    except:
        continue   

try:
    data = pd.concat([exist_data,pd.DataFrame(new_data)]).reset_index(drop=True)
except:
    data = new_data

data[-3:-1]

100%|██████████| 665/665 [3:22:37<00:00, 18.28s/it]  


[{'url': 'http://www.ufcstats.com/fight-details/ccee020be2e8d9f6',
  'event_url': 'http://www.ufcstats.com/event-details/a6a9ab5a824e8f66',
  'date': Timestamp('1994-03-11 00:00:00'),
  'result': 'W',
  'fighter': 'Johnny Rhode',
  'opponent': 'David Levick',
  'title': 'Open Weight Bout',
  'method': 'KO/TKO',
  'round': '1',
  'time': '12:13',
  'format': 'No Time Limit',
  'f_head_str': '9of15',
  'f_body_str': '1of1',
  'f_leg_str': '1of1',
  'f_distance_str': '1of1',
  'f_clinch_str': '1of1',
  'f_ground_str': '9of15',
  'o_head_str': '4of5',
  'o_body_str': '0of0',
  'o_leg_str': '0of0',
  'o_distance_str': '1of2',
  'o_clinch_str': '2of2',
  'o_ground_str': '1of1',
  'f_head_str_perc': 0.81,
  'f_body_str_perc': 0.09,
  'f_leg_str_perc': 0.09,
  'f_dist_str_perc': 0.09,
  'f_clinch_str_perc': 0.09,
  'f_ground_str_perc': 0.81,
  'o_head_str_perc': 1.0,
  'o_body_str_perc': 0.0,
  'o_leg_str_perc': 0.0,
  'o_dist_str_perc': 0.25,
  'o_clinch_str_perc': 0.5,
  'o_ground_str_perc':

In [8]:
data = pd.DataFrame(data) 
data.tail(3)

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_str,f_body_str,f_leg_str,f_distance_str,f_clinch_str,f_ground_str,o_head_str,o_body_str,o_leg_str,o_distance_str,o_clinch_str,o_ground_str,f_head_str_perc,f_body_str_perc,f_leg_str_perc,f_dist_str_perc,f_clinch_str_perc,f_ground_str_perc,o_head_str_perc,o_body_str_perc,o_leg_str_perc,o_dist_str_perc,o_clinch_str_perc,o_ground_str_perc
7236,http://www.ufcstats.com/fight-details/ccee020b...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Johnny Rhode,David Levick,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,9of15,1of1,1of1,1of1,1of1,9of15,4of5,0of0,0of0,1of2,2of2,1of1,0.81,0.09,0.09,0.09,0.09,0.81,1.0,0.0,0.0,0.25,0.5,0.25
7237,http://www.ufcstats.com/fight-details/4b9ae533...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Patrick Smit,Ray Wizar,Open Weight Bout,Submission,1,0:58,No Time Limit,0of0,1of1,0of0,0of0,1of1,0of0,0of0,0of0,1of1,1of1,0of0,0of0,0.00,1.00,0.00,0.00,1.00,0.00,0.0,0.0,1.0,1.00,0.0,0.00
7238,http://www.ufcstats.com/fight-details/4acab678...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Scott Morri,Sean Daughert,Open Weight Bout,Submission,1,0:20,No Time Limit,1of1,0of0,0of0,0of0,1of1,0of0,0of2,0of0,0of2,0of3,0of1,0of0,1.00,0.00,0.00,0.00,1.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00


In [9]:
pd.DataFrame(data).to_csv('data_pre_processed.csv',index=[0])